In [1]:
import paddle
import paddle.nn as nn
from math import ceil

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


## 基础模块
* ConvBN、Swish 和 SE 模块

In [2]:
def ConvBN(out, in_channels, channels, kernel=1, stride=1, pad=0, num_group=1, act=None):
    out.append(nn.Conv2D(in_channels, channels, kernel,
                         stride, pad, groups=num_group, bias_attr=False))
    out.append(nn.BatchNorm2D(channels))
    if act == 'swish':
        out.append(Swish())
    elif act == 'relu':
        out.append(nn.ReLU())
    elif act == 'relu6':
        out.append(nn.ReLU6())


class Swish(nn.Layer):
    def __init__(self):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * nn.functional.sigmoid(x)


class SE(nn.Layer):
    def __init__(self, in_channels, channels, se_ratio=12):
        super(SE, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2D(1)
        self.fc = nn.Sequential(
            nn.Conv2D(in_channels, channels // se_ratio,
                      kernel_size=1, padding=0),
            nn.BatchNorm2D(channels // se_ratio),
            nn.ReLU(),
            nn.Conv2D(channels // se_ratio, channels,
                      kernel_size=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.fc(y)
        return x * y

## LinearBottleneck

In [4]:
class LinearBottleneck(nn.Layer):
    def __init__(self, in_channels, channels, t, stride, use_se=True, se_ratio=12):
        super(LinearBottleneck, self).__init__()
        self.use_shortcut = stride == 1 and in_channels <= channels
        self.in_channels = in_channels
        self.out_channels = channels

        out = []
        if t != 1:
            dw_channels = in_channels * t
            ConvBN(out, in_channels=in_channels,
                   channels=dw_channels, act='swish')
        else:
            dw_channels = in_channels

        ConvBN(out, in_channels=dw_channels, channels=dw_channels, kernel=3, stride=stride, pad=1,
               num_group=dw_channels)

        if use_se:
            out.append(SE(dw_channels, dw_channels, se_ratio))

        out.append(nn.ReLU6())
        ConvBN(out, in_channels=dw_channels, channels=channels)
        self.out = nn.Sequential(*out)

    def forward(self, x):
        out = self.out(x)
        if self.use_shortcut:
            out[:, 0:self.in_channels] += x

        return out

## ReXNet

In [5]:
class ReXNetV1(nn.Layer):
    def __init__(self, input_ch=16, final_ch=180, width_mult=1.0, depth_mult=1.0,
                 class_dim=1000, use_se=True, se_ratio=12, dropout_ratio=0.2):
        super(ReXNetV1, self).__init__()

        layers = [1, 2, 2, 3, 3, 5]
        strides = [1, 2, 2, 2, 1, 2]
        use_ses = [False, False, True, True, True, True]

        layers = [ceil(element * depth_mult) for element in layers]
        strides = sum([[element] + [1] * (layers[idx] - 1)
                       for idx, element in enumerate(strides)], [])
        if use_se:
            use_ses = sum([[element] * layers[idx]
                           for idx, element in enumerate(use_ses)], [])
        else:
            use_ses = [False] * sum(layers[:])
        ts = [1] * layers[0] + [6] * sum(layers[1:])

        self.depth = sum(layers[:]) * 3
        stem_channel = 32 / width_mult if width_mult < 1.0 else 32
        inplanes = input_ch / width_mult if width_mult < 1.0 else input_ch

        features = []
        in_channels_group = []
        channels_group = []

        # The following channel configuration is a simple instance to make each layer become an expand layer.
        for i in range(self.depth // 3):
            if i == 0:
                in_channels_group.append(int(round(stem_channel * width_mult)))
                channels_group.append(int(round(inplanes * width_mult)))
            else:
                in_channels_group.append(int(round(inplanes * width_mult)))
                inplanes += final_ch / (self.depth // 3 * 1.0)
                channels_group.append(int(round(inplanes * width_mult)))

        ConvBN(features, 3, int(round(stem_channel * width_mult)),
               kernel=3, stride=2, pad=1, act='swish')

        for block_idx, (in_c, c, t, s, se) in enumerate(zip(in_channels_group, channels_group, ts, strides, use_ses)):
            features.append(LinearBottleneck(in_channels=in_c,
                                             channels=c,
                                             t=t,
                                             stride=s,
                                             use_se=se, se_ratio=se_ratio))

        pen_channels = int(1280 * width_mult)
        ConvBN(features, c, pen_channels, act='swish')

        features.append(nn.AdaptiveAvgPool2D(1))
        self.features = nn.Sequential(*features)
        self.output = nn.Sequential(
            nn.Dropout(dropout_ratio),
            nn.Conv2D(pen_channels, class_dim, 1))

    def forward(self, x):
        x = self.features(x)
        x = self.output(x).squeeze()
        return x

## 预设模型

In [6]:
def rexnet_100(pretrained=False, **kwargs):
    model = ReXNetV1(width_mult=1.0, **kwargs)
    return model


def rexnet_130(pretrained=False, **kwargs):
    model = ReXNetV1(width_mult=1.3, **kwargs)
    return model


def rexnet_150(pretrained=False, **kwargs):
    model = ReXNetV1(width_mult=1.5, **kwargs)
    return model


def rexnet_200(pretrained=False, **kwargs):
    model = ReXNetV1(width_mult=2.0, **kwargs)
    return model


def rexnet_300(pretrained=False, **kwargs):
    model = ReXNetV1(width_mult=3.0, **kwargs)
    return model

## 数据准备

In [7]:
import paddle.vision.transforms as T
from paddle.vision.datasets import Cifar10

transform = T.Compose([
    T.Resize(size=(224,224)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],data_format='HWC'),
    T.ToTensor()
])

train_dataset = Cifar10(mode='train', transform=transform)
val_dataset = Cifar10(mode='test',  transform=transform)


Cache file /home/aistudio/.cache/paddle/dataset/cifar/cifar-10-python.tar.gz not found, downloading https://dataset.bj.bcebos.com/cifar/cifar-10-python.tar.gz 
Begin to download

Download finished


In [8]:
model=paddle.Model(rexnet_100())
model.summary((1,3,224,224))

W0708 13:16:49.429764   782 device_context.cc:362] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 10.1
W0708 13:16:49.435140   782 device_context.cc:372] device: 0, cuDNN Version: 7.6.


--------------------------------------------------------------------------------
    Layer (type)         Input Shape          Output Shape         Param #    
      Conv2D-1        [[1, 3, 224, 224]]   [1, 32, 112, 112]         864      
   BatchNorm2D-1     [[1, 32, 112, 112]]   [1, 32, 112, 112]         128      
      Swish-1        [[1, 32, 112, 112]]   [1, 32, 112, 112]          0       
      Conv2D-2       [[1, 32, 112, 112]]   [1, 32, 112, 112]         288      
   BatchNorm2D-2     [[1, 32, 112, 112]]   [1, 32, 112, 112]         128      
      ReLU6-1        [[1, 32, 112, 112]]   [1, 32, 112, 112]          0       
      Conv2D-3       [[1, 32, 112, 112]]   [1, 16, 112, 112]         512      
   BatchNorm2D-3     [[1, 16, 112, 112]]   [1, 16, 112, 112]         64       
 LinearBottleneck-1  [[1, 32, 112, 112]]   [1, 16, 112, 112]          0       
      Conv2D-4       [[1, 16, 112, 112]]   [1, 96, 112, 112]        1,536     
   BatchNorm2D-4     [[1, 96, 112, 112]]   [1, 96,

{'total_params': 4838251, 'trainable_params': 4755495}

## 训练自己的模型

In [9]:
model.prepare(optimizer=paddle.optimizer.Adam(learning_rate=0.001,parameters=model.parameters()),
              loss=paddle.nn.CrossEntropyLoss(),
              metrics=paddle.metric.Accuracy())


model.fit(
    train_data=train_dataset, 
    eval_data=val_dataset, 
    batch_size=128, 
    epochs=10, 
    verbose=1, 
)

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/10


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global m

step 391/391 [==============================] - loss: 1.4967 - acc: 0.3725 - 1s/step          
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 79/79 [==============================] - loss: 1.3482 - acc: 0.4607 - 961ms/step          
Eval samples: 10000
Epoch 2/10
step 391/391 [==============================] - loss: 1.1080 - acc: 0.5388 - 1s/step          
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 79/79 [==============================] - loss: 0.9144 - acc: 0.5612 - 965ms/step          
Eval samples: 10000
Epoch 3/10
step 391/391 [==============================] - loss: 0.9853 - acc: 0.6282 - 1s/step          
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 79/79 [==============================] - loss: 1.1932 - acc: 0.6316 - 960ms/step    

In [ ]:
model.save('myReXNet')